# Documentation for the Code Challenges using TiddlyWiki

The notebook aims to convert code solutions into comprehensive documentation stored on a TiddlyWiki-powered webpage. Each solution becomes a dedicated 'tiddler,' providing self-contained implementation details. The seamless integration of these tiddlers enriches TiddlyWiki, offering readers a valuable resource to explore and learn solution concepts conveniently. The TiddlyWiki-based documentation becomes an accessible and informative reference for enthusiasts and learners alike.

**The link to the final wiki containing the documentation can be found [here](https://ai-code-challenge.tiddlyhost.com/).**

![A clever cat stands attentively at the library, engrossed in reading through code documentation.](./code_illustration/tiddlywiki_docs.jpg)

# Leveraging TiddlyWiki for Enhanced Documentation

This project utilizes TiddlyWiki, a unique non-linear note-taking and personal wiki software. It enables efficient information organization in an interconnected and adaptable manner. Unlike traditional wikis, TiddlyWiki operates as a self-contained application within a web browser, using a single HTML file with JavaScript code.

The project leverages TiddlyWiki's flexibility to document code generated in the notebook "Automated Solutions for the Challenges." By doing so, it provides a powerful tool for effective information organization and knowledge management support.

## Accessing the Encrypted File to Obtain the API Key

In [ ]:
%run "secure-api-key-encryption-and-retrieval.ipynb"
password_for_decryption = getpass.getpass("Enter your password to decrypt the API key: ")
openai.api_key = get_decrypted_api_key(password_for_decryption)

## Creating TiddlyWiki Files with Challenge Descriptions

In [ ]:
df = pd.read_csv('code_challenge.csv', delimiter=';', encoding='latin') 

# Drop index column
df = df.reset_index(drop=True) 

beginner_df = df.loc[df['difficulty'] == 'Beginner']
beginner_df.head()

In [ ]:
def generate_problem_enunciation(problem_parameter):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=f"Elaborate what is {problem_parameter}  describing it as a problem enunciation.",
        max_tokens=333,
        n=1
    )
    return response.choices[0].text.strip()

def generate_challenge_tiddlers(tiddlywiki_folder, dataframe):
    
    programming_languages = ['Python', 'JavaScript', 'Java', 'C#', 'C++']
    
    if not os.path.exists(tiddlywiki_folder):
        os.makedirs(tiddlywiki_folder)

    for index, row in dataframe.iterrows():
        problem_id = '{:03d}'.format(row['id'])\n",
        problem_title = row['name']
        difficulty = row['difficulty']
        category = row['category']
        problem_parameter = f' {problem_title} ({category})'
        problem_enunciation = generate_problem_enunciation(problem_parameter)
        
        tiddler_title = f'{problem_id} {problem_title}'
        tiddler_header = f"tags: [[{difficulty}]] [[{category}]] \ntitle: {tiddler_title}\n\n"
        tiddler_enunciation = f"{problem_enunciation}\n\n"
        tiddler_solution = '__Code Solutions__\n\n'
        for language in programming_languages:
            tiddler_solution = tiddler_solution + f"\n* [[{tiddler_title} - {language}]]"
        tiddler_content = tiddler_header + tiddler_enunciation + tiddler_solution
        
        
        tiddler_filename = f"{tiddler_title}.tid" 
        tiddler_path = os.path.join(tiddlywiki_folder, tiddler_filename)

        with open(tiddler_path, 'w') as tiddler_file:
            tiddler_file.write(tiddler_content)
        time.sleep(23)

    print("TiddlyWiki tiddler files generated successfully!")

current_path = os.getcwd()
tiddlywiki_folder = os.path.join(current_path, "code_challenge_documentation", "tiddlywiki")
generate_challenge_tiddlers(tiddlywiki_folder, beginner_df)


## Transforming Code into TiddlyWiki Files

In [ ]:
def generate_tiddlers(folder_path, language, tiddlywiki_folder):
    
    if not os.path.exists(tiddlywiki_folder):
        os.makedirs(tiddlywiki_folder)
    
    file_extension_map = {
        "python": ".py",
        "java": ".java",
        "javascript": ".js",
        "c++": ".cpp",
        "c#": ".cs"
    }

    code_block_map = {
        "python": "Python",
        "java": "Java",
        "javascript": "JavaScript",
        "c++": "C++",
        "c#": "C#"
    }

    for filename in os.listdir(folder_path):
        if filename.endswith(file_extension_map[language]):
            with open(os.path.join(folder_path, filename), 'r') as file:
                content = file.read()

            tiddler_title = f'title:{filename} - {code_block_map[language]}\n'.replace(file_extension_map[language], '')
            tiddler_tag = f"tags: {code_block_map[language]}\n"
            content = content.replace(f"```{code_block_map[language]}", '').replace("```", '')
            tiddler_content = f"{tiddler_title}{tiddler_tag}\n```{code_block_map[language]}\n{content}\n```"
            tiddler_filename = os.path.splitext(filename)[0] + f" - {code_block_map[language]}.tid"
            tiddler_path = os.path.join(tiddlywiki_folder, tiddler_filename)
            
            with open(tiddler_path, 'w') as tiddler_file:
                tiddler_file.write(tiddler_content)
    print(f"TiddlyWiki tiddler files for {language} generated successfully!")

current_path = os.getcwd()
languages = ["python", "java", "javascript", "c++", "c#"]

for lang in languages:
    folder_path = os.path.join(current_path, "code_challenge_solution", lang)
    tiddlywiki_folder = os.path.join(current_path, "code_challenge_documentation", lang)
    generate_tiddlers(folder_path, lang, tiddlywiki_folder)
